In [ ]:
import importlib
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt4
import seaborn as sns

## Explore the amount of nans

In [ ]:
# Explore nans
with sns.plotting_context('paper', font_scale=2): #, rc={"figure.figsize": [6.4,4.4]}):
    file_path = './data/Dane/1year.csv'
    data = pd.read_csv(file_path, na_values='?').drop(['class'],axis=1)
    print('In total we have {} rows and {} features'.format(*data.shape))
    print('Total share of nans is {}'.format(nan_share_tot)) 
    na_data = pd.isnull(data)
    num_nans = na_data.sum(axis=1)
    # Features with most nans
    print((na_data.sum(axis=0)/len(na_data)).sort_values(ascending=False)[:10])
    plt.figure(1)
    plt.hist(na_data.sum(axis=0).drop(['attr37', 'attr21'])/len(na_data)*100, bins=25)
    plt.title('Missing values per feature')
    plt.xlabel('% missing values')
    plt.ylabel('Features')
    savepath = 'hist_nan_features.eps'
    plt.savefig(savepath, format='eps', bbox_inches = 'tight')
    plt.figure(2)
    plt.hist(num_nans, bins=num_nans.max())
    plt.title('Missing values per company')
    plt.xlabel('Number of missing values')
    plt.ylabel('Companies')
    savepath = 'hist_nan_companies.eps'
    plt.savefig(savepath, format='eps', bbox_inches = 'tight')
    plt.show()
    nan_share_tot = num_nans.sum()/float((data.shape[0]*data.shape[1]))

In [ ]:
# What happens if we through away the two features with highest frequency nans?
na_data_dropped_some = na_data.drop(['attr37', 'attr21'], axis=1)
num_nan_dropped = na_data_dropped_some.sum(axis=1)
share_with_nan = (num_nan_dropped != 0).mean()
print('Then we get {} share with at least one nan.'.format(share_with_nan))
data_dropped = data.drop(['attr37', 'attr21'], axis=1)
nan_share_dropped = num_nan_dropped.sum()/float((data_dropped.shape[0]*data_dropped.shape[1]))
print('The total share of nans is then {}'.format(nan_share_dropped))